# Experiment Here!

In [1]:
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import madmom
import librosa
import mir_eval

In [2]:
FPS = 100

In [3]:
from madmom.utils import search_files, match_file

AUDIO_FILES = search_files('data/train', '.flac')

def find_audio_files(ann_files, audio_files, ann_suffix=None, audio_suffix='.flac'):
    """
    Find matching audio files.
    
    Parameters
    ----------
    ann_files : list
        List with annotation file names.
    audio_files : list
        List with audio file names to be matched
    ann_suffix : str, optional
        Suffix of the annotation files. If 'None'
        the suffix is inferred from the annotation
        files.
    audio_suffix : str, optional
        Suffix of the audio files.
    
    Returns
    -------
    matched_files : list
        List of matched audio file (names).
    matched_indices : list
        List of matching indices in `audio_files`.
        
    """
    matched_files = []
    matched_indices = []
    for i, ann_file in enumerate(ann_files):
        if ann_suffix is None:
            ann_suffix = os.path.splitext(ann_file)[1]
        matches = match_file(ann_file, audio_files,
                             ann_suffix, audio_suffix)
        if len(matches) == 1:
            matched_files.append(matches[0])
            matched_indices.append(i)
        else:
            continue
    return matched_files, matched_indices

In [4]:
# CUSTOM

# len(AUDIO_FILES)

# from scripts import utilities
# utilities.main()

# Pre-Processing

## Task: 1

In [5]:
# define additional constants
SR = 44100 # samping rate
FRAME_SIZE = 2048 # number of samples per frame
HOP_SIZE = int(SR / FPS) # hop size depends on sampling rate and frame rate
NUM_BANDS = 40 # number of mel bins

def pre_process(filename, frame_size=2048, frame_rate=FPS, num_bands=40, **kwargs):
    """
    Pre-process the audio signal.

    Parameters
    ----------
    filename : str
        File to be processed.
    frame_size : int
        Size of the frames.
    frame_rate : float
        Frame rate used for the STFT.
    num_bands : int
        Number of frequency bands for the Mel filterbank.
    kwargs : dict, optional
        Additional keyword arguments.

    Returns
    -------
    spectrogram : numpy array
        Spectrogram.

    """    
    # STEP 1: read in audio
    signal, sampling_rate_unused = librosa.load(filename, sr=SR) # read file
    
    # STEP 2,3: compute stft (default windowing function is Hann)
    stft = librosa.core.stft(y=signal, n_fft=frame_size, hop_length=HOP_SIZE)
    
    # STEP 4: discard phase info and square magnitudes
    initial_spectrogram = abs(stft)**2
    
    # STEP 5: apply mel scaling
    mel_bins = librosa.filters.mel(sr=SR, n_fft=frame_size, n_mels=num_bands)
    mel_spectrogram = mel_bins.dot(initial_spectrogram)
    
    # STEP 6: apply DB scaling
    db_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    
    # double check
    # mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr, n_fft=frame_size, hop_length=hop_size, n_mels=num_bands)
    # db_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    # print((db_mel_spectrogram)[0])
        
    spectrogram = db_mel_spectrogram
    return spectrogram

In [6]:
# CUSTOM

from librosa.display import specshow

def test_pre_process():
    texasName = AUDIO_FILES[19] #AUDIO_FILES[19]

    spectrogram = pre_process(texasName, FRAME_SIZE, FPS, NUM_BANDS)

    # print(spectrogram.shape)

    plt.figure(figsize=(15, 5))
    specshow(spectrogram, sr=SR, hop_length=HOP_SIZE, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')

# test_pre_process()

In [7]:
# list for collecting pre-processed spectrograms
# Note: it is not necessary to use this list but recommended in order to
#       avoid recomputation of the same features over and over again.
#       *_AUDIO_IDX canbe used to acces the precomputed spectrograms by
#       index.
SPECTROGRAMS = []

for audio_file in AUDIO_FILES:
    spec = pre_process(audio_file)
    SPECTROGRAMS.append(spec)

# Onset detection

In [8]:
# you are not required to use these predefined constants, but it is recommended
ONSET_ANNOTATION_FILES = search_files('data/train', '.onsets')
ONSET_AUDIO_FILES, ONSET_AUDIO_IDX = find_audio_files(ONSET_ANNOTATION_FILES, AUDIO_FILES)
ONSET_AUDIO = [SPECTROGRAMS[i] for i in ONSET_AUDIO_IDX]
ONSET_ANNOTATIONS = [madmom.io.load_onsets(f) for f in ONSET_ANNOTATION_FILES]

assert len(ONSET_ANNOTATION_FILES) == 321
assert len(ONSET_AUDIO_FILES) == 321
assert len(ONSET_AUDIO) == 321
assert len(ONSET_ANNOTATIONS) == 321

## Task: 2a

In [184]:
def onset_detection_function(spectrogram):
    """
    Compute an onset detection function.

    Parameters
    ----------
    spectrogram : numpy array
        Spectrogram

    Returns
    -------
    odf : numpy array
        Onset detection function.

    """
    spectrogram_T = spectrogram.transpose()
    
    #print(spectrogram_T[100][2], " with shape: ", spectrogram_T.shape)
    #print(spectrogram[2][100])
    
    spectral_flux = []
    for i, frame in enumerate(spectrogram_T):
        sum = 0
        for j, bin in enumerate(frame):
            diff = spectrogram_T[i][j] - (spectrogram_T[i-1][j] if i > 0 else 0)
            print(diff)
            diff = diff if diff >= 0 else 0
            sum = sum + diff
            print(i, ":", j, ":", diff, ":", spectrogram_T[i][j], ":", spectrogram_T[i-1][j])

        spectral_flux.append(diff)
                    
    odf = spectral_flux
    return odf

In [185]:
odf = onset_detection_function(ONSET_AUDIO[19])

odf_lib = librosa.onset.onset_strength(sr=SR, S=ONSET_AUDIO[19])

print('odf_lib:', odf_lib[23], " len: ", len(odf_lib))
print(odf[22])

-18.58116045036862
0 : 0 : 0 : -18.58116045036862 : 2.305496266309595
-24.79264811519051
0 : 1 : 0 : -24.79264811519051 : 0.9261391995622025
-27.854863924423334
0 : 2 : 0 : -27.854863924423334 : -11.09031950594657
-42.030669756525334
0 : 3 : 0 : -42.030669756525334 : -16.62299297607671
-38.498606580058585
0 : 4 : 0 : -38.498606580058585 : -17.298098687815987
-35.75398404018615
0 : 5 : 0 : -35.75398404018615 : -19.847486979437047
-34.71368241721108
0 : 6 : 0 : -34.71368241721108 : -17.044713334310856
-41.09757582773679
0 : 7 : 0 : -41.09757582773679 : -19.138443527177106
-39.98648014287787
0 : 8 : 0 : -39.98648014287787 : -28.255188461337767
-37.54610201284475
0 : 9 : 0 : -37.54610201284475 : -31.91088360831167
-37.70114708749796
0 : 10 : 0 : -37.70114708749796 : -20.330640710838757
-41.2950235882569
0 : 11 : 0 : -41.2950235882569 : -10.503015345353079
-42.41125420358355
0 : 12 : 0 : -42.41125420358355 : -15.303059434690823
-47.723080392174836
0 : 13 : 0 : -47.723080392174836 : -20.5248

7 : 0 : 0 : -19.31761210180664 : -16.909398976896252
-3.562030834588384
7 : 1 : 0 : -21.72867612933019 : -18.166645294741805
0.3488077187729033
7 : 2 : 0.3488077187729033 : -25.97553821119451 : -26.324345929967414
-2.0662356281323504
7 : 3 : 0 : -29.61325726684837 : -27.547021638716018
-4.45531403144512
7 : 4 : 0 : -33.12533110449674 : -28.670017073051618
1.2105830208759372
7 : 5 : 1.2105830208759372 : -29.92477982277649 : -31.135362843652427
-1.0830862478122825
7 : 6 : 0 : -37.9218582030017 : -36.838771955189415
-2.834999784883827
7 : 7 : 0 : -37.65282814581477 : -34.817828360930946
0.46878400125436315
7 : 8 : 0.46878400125436315 : -38.93523991992653 : -39.404023921180894
-2.8267302640614105
7 : 9 : 0 : -45.34151843569547 : -42.51478817163406
-3.2249066013738457
7 : 10 : 0 : -40.59952603438863 : -37.374619433014786
-1.3606121068237726
7 : 11 : 0 : -37.943213325966155 : -36.58260121914238
-1.676918972705323
7 : 12 : 0 : -40.75213484088839 : -39.075215868183065
-2.594321715383103
7 : 13

12 : 29 : 0.8296969759073178 : -57.97573005105513 : -58.80542702696245
0.49969152721099164
12 : 30 : 0.49969152721099164 : -53.127971247870256 : -53.62766277508125
0.19821640413860564
12 : 31 : 0.19821640413860564 : -51.44752613729504 : -51.64574254143365
-0.1984725476796143
12 : 32 : 0 : -58.785928625181754 : -58.58745607750214
0.0
12 : 33 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 34 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 35 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 36 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 37 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 38 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
12 : 39 : 0.0 : -58.80542702696245 : -58.80542702696245
-0.34239170601853175
13 : 0 : 0 : -23.141143756357252 : -22.79875205033872
0.331770098383501
13 : 1 : 0.331770098383501 : -22.206131736956657 : -22.537901835340158
-2.5843906838673014
13 : 2 : 0 : -28.099328727352972 : -25.51493804348567
-3.39183793

19 : 1 : 25.117459197660523 : 3.18885293211172 : -21.9286062655488
25.46580267270508
19 : 2 : 25.46580267270508 : -2.086251685173288 : -27.55205435787837
22.425330304362532
19 : 3 : 22.425330304362532 : -9.53416100466792 : -31.959491309030454
18.652878661923868
19 : 4 : 18.652878661923868 : -15.865533216373333 : -34.5184118782972
13.90119822101569
19 : 5 : 13.90119822101569 : -18.540371309766172 : -32.44156953078186
14.533478953593661
19 : 6 : 14.533478953593661 : -22.6967740178555 : -37.23025297144916
11.831051858260071
19 : 7 : 11.831051858260071 : -30.22697699254012 : -42.05802885080019
3.2165391096849305
19 : 8 : 3.2165391096849305 : -34.91917893639305 : -38.13571804607798
3.078589730743417
19 : 9 : 3.078589730743417 : -39.67604557954552 : -42.75463531028894
3.6533806316945885
19 : 10 : 3.6533806316945885 : -39.80190899755155 : -43.45528962924614
10.46217431811477
19 : 11 : 10.46217431811477 : -33.619033896185904 : -44.08120821430067
14.39398751531536
19 : 12 : 14.39398751531536 : 

25 : 15 : 0 : -30.619750177954185 : -28.351384925377467
-2.395522861642249
25 : 16 : 0 : -38.28755565410323 : -35.89203279246098
-1.1813227355100366
25 : 17 : 0 : -39.31210060735364 : -38.130777871843605
-1.1189437978549108
25 : 18 : 0 : -40.97684357520598 : -39.85789977735107
-0.8587372741124426
25 : 19 : 0 : -43.30883469089642 : -42.450097416783976
-0.658924224512532
25 : 20 : 0 : -47.85691947247342 : -47.19799524796089
-1.4682571880913926
25 : 21 : 0 : -41.31320577098044 : -39.844948582889046
-2.0702883632247335
25 : 22 : 0 : -41.80680997820507 : -39.73652161498033
-1.419369202650607
25 : 23 : 0 : -49.330681344658 : -47.91131214200739
-1.296782228973683
25 : 24 : 0 : -52.497000226675326 : -51.20021799770164
-1.3576900128182317
25 : 25 : 0 : -49.439386178183604 : -48.08169616536537
-1.2170147101611235
25 : 26 : 0 : -48.87574038709813 : -47.65872567693701
-0.5259000876069138
25 : 27 : 0 : -51.82369190591676 : -51.29779181830985
-1.024131384794643
25 : 28 : 0 : -53.44383845282802 : -52

...

954 : 26 : 0 : -44.181584634048335 : -42.9368729432672
-2.3780728300645535
954 : 27 : 0 : -47.85926520087856 : -45.48119237081401
-2.2002433730231914
954 : 28 : 0 : -45.96884708210065 : -43.76860370907746
-1.5184249258023712
954 : 29 : 0 : -48.90369776981988 : -47.38527284401751
-0.7206539833184635
954 : 30 : 0 : -50.78725696203388 : -50.066602978715416
-1.5414639961571268
954 : 31 : 0 : -53.68878487909008 : -52.14732088293295
-0.7870956203992847
954 : 32 : 0 : -58.68596671071684 : -57.898871090317556
-1.8842463475323683
954 : 33 : 0 : -58.80542702696245 : -56.92118067943008
-3.5514432350819476
954 : 34 : 0 : -58.80542702696245 : -55.2539837918805
0.0
954 : 35 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
954 : 36 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
954 : 37 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
954 : 38 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
954 : 39 : 0.0 : -58.80542702696245 : -58.80542702696245
-0.22111977069006095
955 : 0 : 0 : 13.2

961 : 36 : 0 : -58.80542702696245 : -58.17292421075863
0.0
961 : 37 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
961 : 38 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
961 : 39 : 0.0 : -58.80542702696245 : -58.80542702696245
-0.4635227916709841
962 : 0 : 0 : 12.923596908371549 : 13.387119700042533
-0.4118348668689684
962 : 1 : 0 : 19.74508366438306 : 20.15691853125203
-1.295355376719297
962 : 2 : 0 : 1.358748701875776 : 2.654104078595073
0.526344236340373
962 : 3 : 0.526344236340373 : -6.066913414293086 : -6.593257650633459
-1.5926895643623773
962 : 4 : 0 : -8.655470147861081 : -7.062780583498704
-1.3348207787075101
962 : 5 : 0 : -4.412206370763105 : -3.077385592055595
1.3832927856422756
962 : 6 : 1.3832927856422756 : -17.298471769364593 : -18.68176455500687
0.044705938898770725
962 : 7 : 0.044705938898770725 : -15.895327199385694 : -15.940033138284464
-0.4467899644487474
962 : 8 : 0 : -30.879924250768582 : -30.433134286319834
-1.3308641282485496
962 : 9 : 0 : -29.05507425

969 : 10 : 0 : -12.396566899653482 : -11.117648462590935
-1.2225448575941709
969 : 11 : 0 : -16.119163039103487 : -14.896618181509316
-0.4224962001826249
969 : 12 : 0 : -20.604878500329935 : -20.18238230014731
-1.361606975245536
969 : 13 : 0 : -27.982549959680423 : -26.620942984434887
-1.2112232933294464
969 : 14 : 0 : -34.4287318639458 : -33.21750857061635
-1.558940482557965
969 : 15 : 0 : -36.22400995582793 : -34.665069473269966
-1.5301786490559266
969 : 16 : 0 : -34.220899479689436 : -32.69072083063351
-2.182480963921975
969 : 17 : 0 : -31.683797481923605 : -29.50131651800163
-1.51213248332909
969 : 18 : 0 : -36.04225911057844 : -34.53012662724935
-0.4615173853113319
969 : 19 : 0 : -38.994684796527956 : -38.533167411216624
-0.6368607436235862
969 : 20 : 0 : -43.56253589758246 : -42.92567515395887
0.5297400680168636
969 : 21 : 0.5297400680168636 : -45.236441846896795 : -45.76618191491366
0.2909350092567351
969 : 22 : 0.2909350092567351 : -46.818103162071225 : -47.10903817132796
-1.62

976 : 23 : 0 : -47.66420294354781 : -43.840852118754945
0.9673359150356191
976 : 24 : 0.9673359150356191 : -55.161964383649256 : -56.129300298684875
1.0669829288898498
976 : 25 : 1.0669829288898498 : -57.7384440980726 : -58.80542702696245
-0.32925418386843575
976 : 26 : 0 : -58.21435009813668 : -57.88509591426824
0.0
976 : 27 : 0.0 : -58.80542702696245 : -58.80542702696245
-0.21984590553080352
976 : 28 : 0 : -58.26087399541464 : -58.04102808988384
0.0
976 : 29 : 0.0 : -58.80542702696245 : -58.80542702696245
-0.3070786657952027
976 : 30 : 0 : -58.75301393868503 : -58.44593527288983
-1.1201839980063681
976 : 31 : 0 : -58.45462238713325 : -57.33443838912688
0.0
976 : 32 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
976 : 33 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
976 : 34 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
976 : 35 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
976 : 36 : 0.0 : -58.80542702696245 : -58.80542702696245
0.0
976 : 37 : 0.0 : -58.8054270

983 : 33 : 0.31722865901615194 : -45.43212172612991 : -45.74935038514606
-1.7730755634144444
983 : 34 : 0 : -48.93962886455019 : -47.16655330113574
-2.6594877108132238
983 : 35 : 0 : -52.383434351961995 : -49.72394664114877
-8.158905512234675
983 : 36 : 0 : -53.164056675257534 : -45.00515116302286
-7.989548735382904
983 : 37 : 0 : -55.685678320448496 : -47.69612958506559
-2.6025303459048814
983 : 38 : 0 : -58.80542702696245 : -56.20289668105757
0.0
983 : 39 : 0.0 : -58.80542702696245 : -58.80542702696245
-1.4884599340555384
984 : 0 : 0 : 6.303484022750419 : 7.791943956805958
-1.8258582605991105
984 : 1 : 0 : 6.4981279246369805 : 8.323986185236091
-3.813399136179698
984 : 2 : 0 : -10.269489068944221 : -6.456089932764523
-8.378059668628449
984 : 3 : 0 : -17.65647191471761 : -9.278412246089161
-3.18539987282011
984 : 4 : 0 : -15.285833228885611 : -12.100433356065501
-2.4831272289015676
984 : 5 : 0 : -14.19034695228369 : -11.707219723382122
-3.51607172301507
984 : 6 : 0 : -17.6725987668000